In [4]:
import pandas as pd
import numpy as np

# DATEN LADEN

# Gesammelten Chart-Wochen (ca. 21.000 Zeilen) laden
df_charts = pd.read_csv("../data/processed/df_cleaned_full.csv")

# Frisch abgerufenen Spotify-Metadaten laden
df_meta = pd.read_csv("../data/interim/enriched_data.csv")

print(f"Charts: {df_charts.shape[0]} Zeilen")
print(f"Metadaten: {df_meta.shape[0]} Songs")


Charts: 20999 Zeilen
Metadaten: 1516 Songs


In [5]:
# MERGE VORBEREITEN
# In df_charts die URI säubern, um nur die ID zu behalten
# "spotify:track:" aus jedem Eintrag entfernen
df_charts['track_id'] = df_charts['uri'].str.replace('spotify:track:', '', regex=False)

# Kleiner Check, ob es geklappt hat
print("Beispiel ID nach der Bereinigung:")
print(df_charts['track_id'].iloc[0])

Beispiel ID nach der Bereinigung:
3rUGC1vUpkDG9CZFHMur1t


In [6]:
# MERGE (Tabellen zusammenführen)
# 'Left' join nutzen, um alle Chart-Platzierungen zu behalten.
# Die Metadaten werden überall dort angefügt, wo die track_id matcht.
df_final = pd.merge(df_charts, df_meta, on='track_id', how='left', suffixes=('', '_api'))
df_final.head(5)

,chart_week,rank,uri,artist_names,track_name,peak_rank,previous_rank,weeks_on_chart,streams,track_id,track_name_api,artist_id,release_date,explicit,track_popularity,artist_genres,artist_followers,artist_popularity
0,2024-01-04,1,spotify:track:3rUGC1vUpkDG9CZFHMur1t,Tate McRae,greedy,1,14,16,33855816,3rUGC1vUpkDG9CZFHMur1t,greedy,45dkTj5sMRSjrmBSBeiHym,2023-09-15,True,88.0,NaN,9987007.0,91.0
1,2024-01-04,2,spotify:track:0R6NfOiLzLj4O5VbYSJAjf,Xavi,La Diabla,2,17,4,30894083,0R6NfOiLzLj4O5VbYSJAjf,La Diabla,3Me35AWHCGqW4sZ7bWWJt1,2023-11-30,False,75.0,corridos tumbados|corrido|corridos bélicos|sad...,8640060.0,77.0
2,2024-01-04,3,spotify:track:4xhsWYTOGcal8zt0J161CU,Jack Harlow,Lovin On Me,3,19,8,30778444,4xhsWYTOGcal8zt0J161CU,Lovin On Me,2LIk90788K0zvyj2JJVwkJ,2023-11-10,True,80.0,NaN,4430570.0,78.0
3,2024-01-04,4,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,Cruel Summer,2,20,48,30224692,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,06HL4z0CvFAxyc27GXpf02,2019-08-23,False,88.0,NaN,149529339.0,100.0
4,2024-01-04,5,spotify:track:3vkCueOmm7xQDoJ17W1Pm3,Mitski,My Love Mine All Mine,3,18,15,26430016,3vkCueOmm7xQDoJ17W1Pm3,My Love Mine All Mine,2uYWxilOVlUdk4oV9DvwqK,2023-09-15,False,87.0,NaN,11283651.0,81.0


In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21053 entries, 0 to 21052
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   chart_week         21053 non-null  object 
 1   rank               21053 non-null  int64  
 2   uri                21053 non-null  object 
 3   artist_names       21053 non-null  object 
 4   track_name         21053 non-null  object 
 5   peak_rank          21053 non-null  int64  
 6   previous_rank      21053 non-null  int64  
 7   weeks_on_chart     21053 non-null  int64  
 8   streams            21053 non-null  int64  
 9   track_id           21053 non-null  object 
 10  track_name_api     16746 non-null  object 
 11  artist_id          16746 non-null  object 
 12  release_date       16746 non-null  object 
 13  explicit           16746 non-null  object 
 14  track_popularity   16746 non-null  float64
 15  artist_genres      9494 non-null   object 
 16  artist_followers   167

In [9]:
# DATENBEREINIGUNG (CLEANING)

# Spalten löschen, die nicht benötigt werden
cols_to_drop = ['uri', 'track_name_api']
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])
print(f"Spalten {cols_to_drop} gelöscht.")

# Datums-Formate korrigieren
df_final['release_date'] = pd.to_datetime(df_final['release_date'], errors='coerce')
df_final['chart_week'] = pd.to_datetime(df_final['chart_week'], errors='coerce')
print("Datumsspalten konvertiert.")

# Fehlende Werte (NaN) behandeln
# Genres auf 'unknown' setzen
df_final['artist_genres'] = df_final['artist_genres'].fillna('unknown')
print("NaNs behandelt.")

# 4. Check: Chart-Einträge ohne API-Metadaten
missing_meta = df_final['artist_id'].isna().sum()
total_rows = len(df_final)
print(f"Hinweis: {missing_meta} von {total_rows} Zeilen ({missing_meta/total_rows:.1%}) haben keine API-Metadaten erhalten.")

# Endergebnis prüfen
print("\nFinale Spalten-Informationen:")
print(df_final.info())

# Speichern
df_final.to_csv("../data/processed/final_data_with_metadata.csv", index=False, encoding='utf-8')
print("\nFinales Dataset gespeichert.")

Spalten ['uri', 'track_name_api'] gelöscht.
Datumsspalten konvertiert.
NaNs behandelt.
Hinweis: 4307 von 21053 Zeilen (20.5%) haben keine API-Metadaten erhalten.

Finale Spalten-Informationen:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21053 entries, 0 to 21052
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_week         21053 non-null  datetime64[ns]
 1   rank               21053 non-null  int64         
 2   artist_names       21053 non-null  object        
 3   track_name         21053 non-null  object        
 4   peak_rank          21053 non-null  int64         
 5   previous_rank      21053 non-null  int64         
 6   weeks_on_chart     21053 non-null  int64         
 7   streams            21053 non-null  int64         
 8   track_id           21053 non-null  object        
 9   artist_id          16746 non-null  object        
 10  release_date       16458 non-null